In [114]:
import piheaan as heaan
from piheaan.math import sort
from piheaan.math import approx # for piheaan math function
import math
import numpy as np
import pandas as pd
import os

In [115]:
# set parameter
params = heaan.ParameterPreset.FGb
context = heaan.make_context(params) # context has paramter information
heaan.make_bootstrappable(context) # make parameter bootstrapable

# create and save keys
key_file_path = "./keys"
sk = heaan.SecretKey(context) # create secret key
os.makedirs(key_file_path, mode=0o775, exist_ok=True)
sk.save(key_file_path+"/secretkey.bin") # save secret key

key_generator = heaan.KeyGenerator(context, sk) # create public key
key_generator.gen_common_keys()
key_generator.save(key_file_path+"/") # save public key

# load secret key and public key
# When a key is created, it can be used again to save a new key without creating a new one
key_file_path = "./keys"

sk = heaan.SecretKey(context,key_file_path+"/secretkey.bin") # load secret key
pk = heaan.KeyPack(context, key_file_path+"/") # load public key
pk.load_enc_key()
pk.load_mult_key()

eval = heaan.HomEvaluator(context,pk) # to load piheaan basic function
dec = heaan.Decryptor(context) # for decrypt
enc = heaan.Encryptor(context) # for encrypt

In [116]:
def calculate_result(user_id,df):
    # Function for making the dictionary by User ID
    
    def making_dict(column_name):
        result = df.groupby('User ID')[column_name].apply(list).reset_index()
        user_dictionary = {}
        for index, row in result.iterrows():
            user_dictionary[row[0]] = row[1]
        return user_dictionary

    # price is Average Price
    user_dict_price = making_dict('price')
    user_dict_price_diff = making_dict('평균 가격 대비')
    user_dict_qty = making_dict('qty')

    # To decide log_slots and num_slots, find the max length of the user's data
    temp = []
    for i in user_dict_qty.keys():
        temp.append(len(user_dict_qty[i]))
    max_len = max(temp)
    max_len

    # Find the most near 2 powered number of max_len
    log_slots = 0
    num_slots = 2**log_slots
    while max_len > num_slots:
        log_slots += 1
        num_slots = 2**log_slots

    data1 = user_dict_qty[user_id]
    actual_len = len(data1)
    data1_padded = [0] * num_slots
    for i in range(len(data1)):
        data1_padded[i] = data1[i]
        
    message_1 = heaan.Message(log_slots)
    for i in range(num_slots):
        message_1[i] = data1_padded[i]

    data2 = user_dict_price_diff[user_id]
    data2_padded = [0] * num_slots
    for i in range(len(data2)):
        data2_padded[i] = data2[i]
        
    message_2 = heaan.Message(log_slots)
    for i in range(num_slots):
        message_2[i] = data2_padded[i]
        
    data3 = user_dict_price[user_id]
    data3_padded = [0] * num_slots
    for i in range(len(data3)):
        data3_padded[i] = data3[i]

    message_3 = heaan.Message(log_slots)
    for i in range(num_slots):
        message_3[i] = data3_padded[i]

    ciphertext_1 = heaan.Ciphertext(context) # qty
    ciphertext_2 = heaan.Ciphertext(context) # price difference 현재 가격과 평균 가격 차이
    ciphertext_3 = heaan.Ciphertext(context) # price 평균 가격

    enc.encrypt(message_1, pk, ciphertext_1)
    enc.encrypt(message_2, pk, ciphertext_2)
    enc.encrypt(message_3, pk, ciphertext_3)

    # (ciphertext * ciphertext)
    result_mult = heaan.Ciphertext(context)
    eval.mult(ciphertext_1, ciphertext_2, result_mult)

    result_mult_message = heaan.Message(log_slots)
    dec.decrypt(result_mult, sk, result_mult_message)

    # (ciphertext * ciphertext)
    result_mult2 = heaan.Ciphertext(context)
    eval.mult(ciphertext_1, ciphertext_3, result_mult2)

    result_mult_message2 = heaan.Message(log_slots)
    dec.decrypt(result_mult2, sk, result_mult_message2)

    data = [0.01] * num_slots
    message_01 = heaan.Message(log_slots)
    for i in range(num_slots):
        message_01[i] = data[i]

    ciphertext_4 = heaan.Ciphertext(context) # 0.1만 있는거
    enc.encrypt(message_01, pk, ciphertext_4)

    # (ciphertext * ciphertext)
    result_mult3 = heaan.Ciphertext(context)
    eval.mult(ciphertext_4, result_mult2, result_mult3)

    result_mult_message3 = heaan.Message(log_slots)
    dec.decrypt(result_mult3, sk, result_mult_message3)

    data = result_mult_message3
    message = heaan.Message(log_slots)
    for i in range(num_slots):
        message[i] = data[i]
        
    ciphertext = heaan.Ciphertext(context)
    result_inv = heaan.Ciphertext(context)

    enc.encrypt(message, pk, ciphertext)
    approx.inverse(eval, ciphertext, result_inv) 

    decryptor = heaan.Decryptor(context)
    result_inv_message = heaan.Message(log_slots)

    decryptor.decrypt(result_inv, sk, result_inv_message)

    # (message * message)
    result_final_message = heaan.Message(log_slots)
    eval.mult(result_inv_message, result_mult_message, result_final_message)

    # (message * message)
    result_final_message2 = heaan.Message(log_slots)
    eval.mult(result_final_message, message_01, result_final_message2)

    final_result= [] # 한 유저의 각 주식별 수익률
    for i in range(actual_len):
        final_result.append(result_final_message2[i])

    total_result = sum(result_mult_message)/sum(result_mult_message2) # 한 유저의 전체 수익률

    return final_result, total_result

In [117]:
file_path = 'trader_list_1.csv'
df1 = pd.read_csv(file_path)

file_path = 'trader_list_2.csv'
df2 = pd.read_csv(file_path)

file_path = 'trader_list_3.csv'
df3 = pd.read_csv(file_path)

In [118]:
# 데이터프레임을 행 단위로 결합
combined_df = pd.concat([df1, df2, df3], ignore_index=True)

In [119]:
# userid의 고유값 확인
unique_userids = combined_df['User ID'].unique()
company_combined = []

In [120]:
for name in unique_userids:
    stock_profit, total_profit = calculate_result(name,combined_df)
    company_combined.append([name, total_profit])
company_combined.sort(key = lambda x:x[0])
company_combined

[['alexwilson3@yahoo.com', (-0.0027369952813597487+0j)],
 ['allisonnelson@gmail.com', (-0.015010722983036682+0j)],
 ['amandabrown@yahoo.com', (0.07262859854175498+0j)],
 ['andrewyoung@gmail.com', (0.011180409264520214+0j)],
 ['angelaroberts@naver.com', (-0.07089304686759733+0j)],
 ['ashleylopez@yahoo.com', (0.06599136695743515+0j)],
 ['ashleyrobinson@gmail.com', (-0.07638476715247416+0j)],
 ['bobbyking@gmail.com', (-0.04950639199568366+0j)],
 ['brandonallen@gmail.com', (0.011180409264520214+0j)],
 ['briancampbell@gmail.com', (0.055222893236190965+0j)],
 ['brianlee@naver.com', (-0.03872975602860862+0j)],
 ['caroladams@gmail.com', (-0.07638476715247416+0j)],
 ['chloebennett@gmail.com', (-0.01594060624808634+0j)],
 ['chrisr45@naver.com', (0.029306095043316653+0j)],
 ['danielkim@gmail.com', (0.1122956137900506+0j)],
 ['davidevans@naver.com', (0.029306095043316653+0j)],
 ['davidnguyen@yahoo.com', (0.06507671378031897+0j)],
 ['emilyjones@gmail.com', (-0.01594060624808634+0j)],
 ['emilyward@n

In [121]:
length = len(company_combined)
new_data = []
for values in company_combined:
    new_data.append(values[1])

In [ ]:
# Find the most near 2 powered number of max_len
log_slots = 0
num_slots = 2**log_slots
while length > num_slots:
    log_slots += 1
    num_slots = 2**log_slots
    
new_data_padded = [0] * num_slots
for i in range(len(new_data)):
    new_data_padded[i] = new_data[i]
    
new_message = heaan.Message(log_slots)

for i in range(num_slots):
    new_message[i]=new_data_padded[i]

new_ciphertext = heaan.Ciphertext(context)
enc.encrypt(new_message, pk, new_ciphertext)

# Input range : -0.5 ~ 0.5

ciphertext_out_sort = heaan.Ciphertext(context)
sort.sort(eval, new_ciphertext, ciphertext_out_sort, num_slots, False) # 4th boolean: 0: descending, 1 : ascending order

message_out_sort = heaan.Message(log_slots)

dec.decrypt(ciphertext_out_sort, sk, message_out_sort)

# print("sort : ", message_out_sort)
profit_rank = []
for i in range(64):
    if i >= 33 and i <= 42: # the values that have 0 need to be deleted
        continue
    profit_rank.append(message_out_sort[i])

index pair in unitSort :0, 1
index pair in unitSort :1, 2
index pair in unitSort :1, 1
index pair in unitSort :2, 4
index pair in unitSort :2, 2
index pair in unitSort :2, 1
index pair in unitSort :3, 8
index pair in unitSort :3, 4
index pair in unitSort :3, 2
index pair in unitSort :3, 1
index pair in unitSort :4, 16
index pair in unitSort :4, 8
index pair in unitSort :4, 4
index pair in unitSort :4, 2
index pair in unitSort :4, 1
index pair in unitSort :5, 32
index pair in unitSort :5, 16
index pair in unitSort :5, 8
index pair in unitSort :5, 4
index pair in unitSort :5, 2
index pair in unitSort :5, 1


# Profit Rank 
### 1 matthewmiller@naver.com
### 2 danielkim@gmail.com
### 3 emilyward@naver.com, lisajackson@naver.com

In [124]:
profit_rank

[(0.14714149318562905+0j),
 (0.11229561379005144+0j),
 (0.08516291589956171+0j),
 (0.08516291589956171+0j),
 (0.07262859854175478+0j),
 (0.06599136695743447+0j),
 (0.06599136695743447+0j),
 (0.06599136695743447+0j),
 (0.06507671378031867+0j),
 (0.06507671378031867+0j),
 (0.06507671378031867+0j),
 (0.05522289323619128+0j),
 (0.05522289323619128+0j),
 (0.045852622559441426+0j),
 (0.045852622559441426+0j),
 (0.045852622559441426+0j),
 (0.045702875747765354+0j),
 (0.045702875747765354+0j),
 (0.043386839392379285+0j),
 (0.04010544875882155+0j),
 (0.04010544875882155+0j),
 (0.04010544875882155+0j),
 (0.03776972417363532+0j),
 (0.02930609504331689+0j),
 (0.02930609504331689+0j),
 (0.02930609504331689+0j),
 (0.013382897622586721+0j),
 (0.013382897622586721+0j),
 (0.011180409264520226+0j),
 (0.011180409264520226+0j),
 (0.006052908219404626+0j),
 (0.006052908219404626+0j),
 (0.0060529082194045545+0j),
 (-0.0027369952813597843+0j),
 (-0.015010722983036523+0j),
 (-0.015010722983036523+0j),
 (-0.01